In [55]:
import math
import numpy as np
from time import time
import pandas as pd

"""
m is a resultant brightness
m$_{\odot}$ is a Sun’s magnitude
$\varphi$ is the satellite-sun-sensor solar phase angle (0º < STO < 90º)
A is the cross-sectonal area
$\textcolor{blue}{\rho}$ is the bond albedo
R$\textcolor{orange}{_{m}}$= is the range to target (400km < R$\textcolor{orange}{_m}$ < 2500 km) 
"""

mag_v_sun = -26.72              #Sun's V magnitude of Gaia   Jordi et al. 2010

In [ ]:
### TEST
model_magnitud(-26.95, 24.5, 1, 0.5, 18423.2, 1)

### Model of brightness

In [56]:
def model_magnitud(data_in, parameters_fit):
	mag_v_sun = -26.72

	phase_angle = data_in[1]
	range_sat = data_in[2]
	cross_section = parameters_fit[0]
	albedo = parameters_fit[1] 
	mix_coe = parameters_fit[2]

   # Change degrees to radian
	degtorad = lambda x : x*np.pi/180.

   # Definition coefficients of equation
	coe_1 = 2/(3*np.pi)
	coe_2 = 1/(4*np.pi)

	phase_angle_rad = degtorad(phase_angle)

	part1 = (coe_1 *((np.pi - phase_angle_rad)*np.cos(phase_angle_rad) - np.sin(phase_angle_rad))) - coe_2
	print(part1)
	mag = mag_v_sun - 2.5 * np.log10(cross_section * albedo * (mix_coe * part1) + coe_2) + 5*np.log10(range_sat)
	
	data_out = {'mag' : mag,
			'phase_angle' : phase_angle,
			'cross_sectio' : cross_section,
			'albedo' : albedo,
			'range_sat' : range_sat,
			'mix_coe' : mix_coe
	}
	
	# all_mag = pd.DataFrame(data_out)
	# print(data_out)
	return mag

### Define model and sum-of-squares function 

theta is a result of mcstat = MCMC()

In [57]:
def sum_squares(data, theta):

    value_model = model_magnitud(data, theta)
    residual = data[0] - value_model
    # print(residual)
    return (residual ** 2).sum(axis=0)
    

In [35]:
path = '/home/kero/Documents/PhD/Re-analyse/result_analyse.csv'
data_sat = pd.read_csv(path)



In [53]:
data_in = [24.5, 18423.2]
parameters_fit = [1., 0.5, .5]
print(data_in[1])
model_magnitud(data_in, parameters_fit)

sum_squares(data_in, parameters_fit)


18423.2
0.3564921766929614
{'mag': -3.460965691709177, 'phase_angle': 24.5, 'cross_sectio': 1.0, 'albedo': 0.5, 'range_sat': 18423.2, 'mix_coe': 0.5}
0.3564921766929614
{'mag': -3.460965691709177, 'phase_angle': 24.5, 'cross_sectio': 1.0, 'albedo': 0.5, 'range_sat': 18423.2, 'mix_coe': 0.5}
-0.5390343082908231


0.29055798551456613

## Monte Carlo

In [58]:
import math
import pymcmcstat
import numpy as np
import pandas as pd
from pymcmcstat.MCMC import MCMC
import matplotlib.pyplot as plt


In [59]:
path = '/home/kero/Documents/PhD/Re-analyse/result_analyse.csv'
data_sat = pd.read_csv(path)

In [60]:
# Constants
# Sun's magnitud
mag_sun = -26.72

# Define data
range_sat = data_sat['range_sat'].to_numpy()
mag_observation = data_sat['mag_observation'].to_numpy()
mag_estimate = data_sat['mag_estimate'].to_numpy()
sat_phase_angle = data_sat['obs_phase_angle'].to_numpy()


data_in = np.array([mag_estimate, sat_phase_angle, range_sat])



In [61]:
# Initialize MCMC object
mcstat = MCMC()

# initialize data structure 
mcstat.data.add_data_set(
    data_in, user_defined_object=model_magnitud
)


TypeError: DataStructure.add_data_set() missing 1 required positional argument: 'y'

In [ ]:
# initialize parameter array
cross_section_int = [0.5, 1.3]
albedo_int = [0., 1.]
mix_coeff_int = [.5, 1.]

mcstat.parameters.add_model_parameter(name='cross_section',
                                      theta0=cross_section_int[0], 
                                      minimum=0.5, maximum=1.3, 
                                      prior_mu=0.9, prior_sigma=0.4)

mcstat.parameters.add_model_parameter(name='albedo',
                                      theta0=albedo_int[0], minimum=0., 
                                      maximum=1., prior_mu=0.5, 
                                      prior_sigma=0.3)

mcstat.parameters.add_model_parameter(name='albedo',
                                      theta0=mix_coeff_int[0], minimum=0.5, 
                                      maximum=1., prior_mu=0.75, 
                                      prior_sigma=0.2)

# Generate samples
mcstat.simulation_options.define_simulation_options(adaptint=2.0e3)

# Define method for estimating the error variance and computing the covariance matrix
mcstat.model_settings.define_model_settings(sos_function = sum_squares)

In [ ]:
# Run mcmcrun
mcstat.run_simulation()

In [ ]:
import math
import numpy as np
from numpy.random import normal
from time import time
import matplotlib.pyplot as plt
import seaborn as sns

cross_section_0 = 0.9
albedo_0 = 0.5

cross_dist = normal(loc=0.9, scale=0.4, size=2000)
albedo = normal(loc=0.5, scale=0.25, size=2000)
sns.histplot(albedo, kde=True)